# Automatically segment customers using purchase data 

In [2]:
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

### Why care about customer segmentation? 

It can provide insights into your customers behavior, habits, and preferences, allowing you to offer tailored marketing campaigns, increasing your odds of success as well as improving customer experience with tailored content. 

### What are we building?

Using transactional purchasing data, we will create a 2 x 2 matrix to create 4 customers groups. Each group will defer from the other, depending from the on the 2 dimensions: 1) current customer value, and 2) potential customer value. 

### What technique are we going to use? 

RFM model: 
- Recency: When was the last time they purchased 
- Frequency: How often and for how long have they purchased?
- Monetary value/sales: How much have they purchased? 

Generally used to identify the highest vale customer at the intersection of each 3 question. To build the 2 x 2 Matrix we'll only use the R and the M from RFM.

### What data are we using?

We are using purchased sample data provided by tableau known as "Global Superstore". It's often used in forecasting and time series analysis. It contains more than 1500 different customers and 4 years of purchase data. Since we're doing a behavioral segmentation and not a demographic segmentation, we will remove some potential demographic bias by filtering only the B2C segment (consumer) and the USA country.



# 1. Load , filter, clean and aggregate the data at the customer level 

In [ ]:
url = 'https://github.com/tristanga/Data-Analysis/raw/master/Global%20Superstore.xls'
df = pd.read_excel(url)
df = df[(df.Segment == 'Consumer') & (df.Country == 'United States')]

In [4]:
df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
9,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,...,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tabl...,4297.644,13,0.4,-1862.3124,865.74,Critical


# 2. Create RFM Features for each customers

In [5]:
df_RFM = df.groupby('Customer ID').agg({'Order Date': lambda y:(df['Order Date'].max().date() - y.max().date()).days, 
                                       'Order ID': lambda y: len(y.unique()), 
                                        'Sales': lambda y:round(y.sum(),2)
                                       })



In [7]:
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
RB-19360,96,6,15117.34
AB-10105,41,10,14473.57
KL-16645,47,12,14175.23
SC-20095,349,9,14142.33
HL-15040,43,6,12873.30


# 3. To automate the segmentation we will use 80% quantile for Recency and Monetary

In [8]:
# We will use the 80% quantile for each feature
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)

df_RFM['R'] = np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F'] = np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M'] = np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)

df_RFM.head()

     Recency  Frequency  Monetary
0.8    222.2        8.0   4070.17


,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
RB-19360,96,6,15117.34,2,1,2
AB-10105,41,10,14473.57,2,2,2
KL-16645,47,12,14175.23,2,2,2
SC-20095,349,9,14142.33,1,2,2
HL-15040,43,6,12873.30,2,1,2


# 4. Calculate RFM score and sort customers 


In [13]:
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str) + df_RFM.R.map(str)

df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'Customer ID': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.head()

,Customer ID,Frequency,Recency,R,M,Monetary
RMScore,,,,,,
22,73,8,59,2,2,6711.0
21,9,7,425,1,2,8564.0
12,254,6,70,2,1,1902.0
11,73,4,440,1,1,1526.0


# 5. Visualize the Value Matrix and explore some key numbers

In [18]:
# 1) Average Monetary Matrix
df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
print('$$')
display(df_RFM_M)


# 2) Number of Customer Matrix
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='Customer ID')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
print('Number of Customers')
display(df_RFM_C)

# 3) Recency Matrix
df_RFM_R = df_RFM_SUM.pivot(index='M', columns='R', values='Recency')
df_RFM_R= df_RFM_R.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
print('Recency')
display(df_RFM_R)

$$


R,1,2
M,,
2,8564.0,6711.0
1,1526.0,1902.0


Number of Customers


R,1,2
M,,
2,9,73
1,73,254


Recency


R,1,2
M,,
2,425,59
1,440,70


### Takeaways
- There are few customers in the "disengaged" bucket (top left matrix). Since there are a few of them, it should be easy to partner with the business to understand what happened at the customer level. Based on the analysis, there might be a simple quick win. 
- The avg. last order from the "Light" bucket i.e. bottom left matrix, is very old (more than a year, vs. 60-70 days for 'engaged' customers). Launching a simple reactivation campaign with a coupon might be an initiative that could lead to some new orders and help some of these customers move to the "New" bucket i.e. bottom right. 